In [1]:
import numpy as np
from sklearn import neighbors
import ot
import joblib
import jpype
import jpype.imports
from jpype.types import *
jpype.startJVM(classpath=['../GOW.jar'])

In [2]:
X_train = joblib.load('../Datasets/SpokenArabicDigit/Cut/X_train.pkl')
X_test = joblib.load('../Datasets/SpokenArabicDigit/Cut/X_test.pkl')
y_train = joblib.load('../Datasets/SpokenArabicDigit/Cut/y_train.pkl')
y_test = joblib.load('../Datasets/SpokenArabicDigit/Cut/y_test.pkl')

print("Train: ", len(X_train))
print("Test: ", len(X_test))

Train:  330
Test:  110


In [3]:
GOW = JClass("GOW")

In [18]:
LAMBDA1 = 15
LAMBDA2 = 30
metric = 'sqeuclidean'

def new_metric(a, seq1, seq2):
    C = ot.dist(seq1, seq2, metric=metric)
    C = C / C.max()
    m = len(seq2)
    b = np.ones(m) / m

    GOOW = GOW(a, b, C, 10, LAMBDA1, LAMBDA2, 0.0001, 100, "w_slope")

    return GOOW.GOW_distance_sinkhorn_autoscale()

In [19]:
X_computed = np.ones((len(X_train), len(X_train)))

X_test_computed = np.empty((len(X_test), len(X_train)))

for i in range(len(X_test)):
    print('Computing:', i)
    seq_1 = np.array(X_test[i])
    n = len(seq_1)
    a = np.ones(n) / n    

    for j in range(len(X_train)):
        X_test_computed[i][j] = new_metric(a, seq_1, np.array(X_train[j]))
    
    print('Done')

# 96m

Computing: 0
Done
Computing: 1
Done
Computing: 2
Done
Computing: 3
Done
Computing: 4
Done
Computing: 5
Done
Computing: 6
Done
Computing: 7
Done
Computing: 8
Done
Computing: 9
Done
Computing: 10
Done
Computing: 11
Done
Computing: 12
Done
Computing: 13
Done
Computing: 14
Done
Computing: 15
Done
Computing: 16
Done
Computing: 17
Done
Computing: 18
Done
Computing: 19
Done
Computing: 20
Done
Computing: 21
Done
Computing: 22
Done
Computing: 23
Done
Computing: 24
Done
Computing: 25
Done
Computing: 26
Done
Computing: 27
Done
Computing: 28
Done
Computing: 29
Done
Computing: 30
Done
Computing: 31
Done
Computing: 32
Done
Computing: 33
Done
Computing: 34
Done
Computing: 35
Done
Computing: 36
Done
Computing: 37
Done
Computing: 38
Done
Computing: 39
Done
Computing: 40
Done
Computing: 41
Done
Computing: 42
Done
Computing: 43
Done
Computing: 44
Done
Computing: 45
Done
Computing: 46
Done
Computing: 47
Done
Computing: 48
Done
Computing: 49
Done
Computing: 50
Done
Computing: 51
Done
Computing: 52
Done
Com

In [20]:
joblib.dump(X_test_computed, "Saved_distance_matrices/%s/gow_dist_sinkhorn_autoscale_LAMBDA1_%s_LAMBDA2_%s.pkl" % (metric, LAMBDA1, LAMBDA2))

['Saved_distance_matrices/sqeuclidean/gow_dist_sinkhorn_autoscale_LAMBDA1_15_LAMBDA2_30.pkl']

In [21]:
num_neighbor = 1
clf = neighbors.KNeighborsClassifier(n_neighbors = num_neighbor, metric="precomputed")
clf.fit(X_computed, y_train)
y_pred = clf.predict(X_test_computed)

print("Predicted labels: ", y_pred)
print("Ground truth    : ", y_test)

from sklearn.metrics import accuracy_score
print("Accuracy of " + str(num_neighbor) + "NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Predicted labels:  [0 0 0 0 0 0 0 7 7 0 0 1 1 1 1 1 1 1 1 1 1 3 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 7 7 7 4 7 4 7 7 7 7 7 3 3 3 3 3 3 3 3 3 3 5 6 6 6 6 7 0 0 6
 6 6 6 7 7 7 7 7 7 7 7 4 7 7 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9]
Ground truth    :  [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6
 6 6 6 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9]
Accuracy of 1NN: 76.36 %


In [22]:
num_neighbor = 2
clf = neighbors.KNeighborsClassifier(n_neighbors = num_neighbor, metric="precomputed")
clf.fit(X_computed, y_train)
y_pred = clf.predict(X_test_computed)

print("Predicted labels: ", y_pred)
print("Ground truth    : ", y_test)

from sklearn.metrics import accuracy_score
print("Accuracy of " + str(num_neighbor) + "NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Predicted labels:  [0 0 0 0 0 0 0 7 7 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 7 7 7 4 7 4 7 7 4 7 7 3 3 3 3 3 3 3 3 3 3 5 6 6 6 6 0 0 0 6
 0 6 6 7 7 7 7 7 7 7 7 4 7 7 8 8 8 8 8 8 8 8 8 3 8 9 9 9 9 9 9 9 9 9 9 9]
Ground truth    :  [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6
 6 6 6 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9]
Accuracy of 2NN: 76.36 %


In [23]:
num_neighbor = 3
clf = neighbors.KNeighborsClassifier(n_neighbors = num_neighbor, metric="precomputed")
clf.fit(X_computed, y_train)
y_pred = clf.predict(X_test_computed)

print("Predicted labels: ", y_pred)
print("Ground truth    : ", y_test)

from sklearn.metrics import accuracy_score
print("Accuracy of " + str(num_neighbor) + "NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Predicted labels:  [0 0 7 0 7 0 0 7 7 7 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 7 7 7 4 7 7 7 7 7 7 7 3 3 3 3 3 3 3 3 3 3 5 6 6 6 6 0 0 0 0
 6 6 6 7 7 7 7 7 7 7 7 4 7 7 8 8 8 8 8 8 8 8 8 3 8 9 9 9 9 9 9 9 9 9 9 9]
Ground truth    :  [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6
 6 6 6 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9]
Accuracy of 3NN: 71.82 %


In [24]:
num_neighbor = 4
clf = neighbors.KNeighborsClassifier(n_neighbors = num_neighbor, metric="precomputed")
clf.fit(X_computed, y_train)
y_pred = clf.predict(X_test_computed)

print("Predicted labels: ", y_pred)
print("Ground truth    : ", y_test)

from sklearn.metrics import accuracy_score
print("Accuracy of " + str(num_neighbor) + "NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Predicted labels:  [0 0 7 0 7 0 0 7 7 7 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 7 7 7 4 7 7 7 4 7 7 7 3 3 3 3 3 3 3 3 3 3 5 6 6 6 6 7 7 0 0
 0 6 6 7 7 7 7 7 7 7 7 4 7 7 8 8 8 8 8 8 8 8 8 3 3 9 9 9 9 9 9 9 9 9 9 9]
Ground truth    :  [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6
 6 6 6 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9]
Accuracy of 4NN: 70.91 %


In [25]:
num_neighbor = 5
clf = neighbors.KNeighborsClassifier(n_neighbors = num_neighbor, metric="precomputed")
clf.fit(X_computed, y_train)
y_pred = clf.predict(X_test_computed)

print("Predicted labels: ", y_pred)
print("Ground truth    : ", y_test)

from sklearn.metrics import accuracy_score
print("Accuracy of " + str(num_neighbor) + "NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Predicted labels:  [0 0 7 0 7 0 0 7 7 7 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 7 7 7 4 7 7 7 4 7 7 7 3 3 3 3 3 3 3 3 3 3 5 6 0 6 6 0 7 0 0
 0 0 6 7 7 7 7 7 7 7 7 4 7 7 8 8 8 8 8 8 8 8 8 3 3 9 9 9 9 9 9 9 9 9 9 9]
Ground truth    :  [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6
 6 6 6 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9]
Accuracy of 5NN: 69.09 %


In [26]:
num_neighbor = 6
clf = neighbors.KNeighborsClassifier(n_neighbors = num_neighbor, metric="precomputed")
clf.fit(X_computed, y_train)
y_pred = clf.predict(X_test_computed)

print("Predicted labels: ", y_pred)
print("Ground truth    : ", y_test)

from sklearn.metrics import accuracy_score
print("Accuracy of " + str(num_neighbor) + "NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Predicted labels:  [0 0 0 0 7 0 0 7 7 7 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 7 7 7 4 4 7 7 4 7 7 7 3 3 3 3 3 3 3 3 3 7 5 6 6 6 6 7 7 0 0
 0 0 6 7 7 7 7 7 7 7 7 4 7 7 8 8 8 8 8 8 8 8 8 3 8 9 9 9 9 9 9 9 9 9 9 9]
Ground truth    :  [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6
 6 6 6 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9]
Accuracy of 6NN: 72.73 %


In [27]:
num_neighbor = 7
clf = neighbors.KNeighborsClassifier(n_neighbors = num_neighbor, metric="precomputed")
clf.fit(X_computed, y_train)
y_pred = clf.predict(X_test_computed)

print("Predicted labels: ", y_pred)
print("Ground truth    : ", y_test)

from sklearn.metrics import accuracy_score
print("Accuracy of " + str(num_neighbor) + "NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Predicted labels:  [0 0 7 0 7 0 0 7 7 7 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 7 7 7 4 4 7 7 4 7 7 7 3 3 3 3 3 3 3 3 3 7 5 6 6 6 6 7 7 0 0
 0 0 6 7 7 7 7 7 7 7 7 4 7 7 8 8 8 8 8 8 8 8 8 8 3 9 9 9 9 9 9 9 9 9 9 9]
Ground truth    :  [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6
 6 6 6 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9]
Accuracy of 7NN: 71.82 %


In [28]:
num_neighbor = 8
clf = neighbors.KNeighborsClassifier(n_neighbors = num_neighbor, metric="precomputed")
clf.fit(X_computed, y_train)
y_pred = clf.predict(X_test_computed)

print("Predicted labels: ", y_pred)
print("Ground truth    : ", y_test)

from sklearn.metrics import accuracy_score
print("Accuracy of " + str(num_neighbor) + "NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Predicted labels:  [0 0 7 0 7 0 0 7 7 7 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 7 7 7 4 4 7 7 4 4 7 7 3 3 3 3 3 3 3 3 3 7 5 6 6 6 6 7 7 0 0
 0 0 6 7 7 7 7 7 7 7 7 4 7 7 8 8 8 8 8 8 8 8 8 8 3 9 9 9 9 9 9 9 9 9 9 9]
Ground truth    :  [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6
 6 6 6 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9]
Accuracy of 8NN: 72.73 %


In [29]:
y_pred

array([0, 0, 7, 0, 7, 0, 0, 7, 7, 7, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       7, 7, 7, 4, 4, 7, 7, 4, 4, 7, 7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 5,
       6, 6, 6, 6, 7, 7, 0, 0, 0, 0, 6, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])